## Skin Color Model
### Goal
The objective of this notebook is to start training on my images using google net's pretrained model. 
These pre trained weights can be easily taken from keras' applications library.



In [1]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import keras
import numpy as np
import PIL
import os
from PIL import Image
import glob
from libs import cnn
from libs import img_processing as im_p
import cv2
import pickle
import pandas as pd

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


In [2]:
train_img_paths = glob.glob('img/train/*')
test_img_paths = glob.glob('img/test/*')

In [3]:
BATCH_SIZE = 128
NB_EPOCH = 20

In [4]:
imp = im_p.ImageProcessor('static/face.xml')

### Image Processing
For each image, I use open cv to detect the bounds of the faces. I then take the upper left corner of the photo to get te skin color.
#### Train Images

In [11]:
X_img = []
y_col = []
for path in train_img_paths:
    img = imp.get_image(path)
    face = imp.crop_face(img, path, 25)
    if face:
        color = imp.get_color(img)
        img_arr = imp.img_to_arr(face)
        X_img.append(img_arr)
        y_col.append(color)
    else:
        print('No face detected in {}'.format(path))

#### Test Images

In [12]:
X_img_test = []
y_col_test = []
for path in test_img_paths:
    img = imp.get_image(path)
    face = imp.crop_face(img, path, 25)
    if face:
        color = imp.get_color(img)
        img_arr = imp.img_to_arr(face)
        x_img_test.append(img_arr)
        y_col_test.append(color)
    else:
        print('No face detected in {}'.format(path))

In [16]:
# turn it to numpy arrays
X_img = np.array(X_img)
y_col = np.array(y_col)
X_img_test = np.array(X_img_test)
y_col_test = np.array(y_col_test)

In [5]:
# save this 
# pickle.dump(X_img, open('states/x_img.pkl', 'wb'))
# pickle.dump(y_col, open('states/y_col.pkl', 'wb'))
# pickle.dump(X_img_test, open('states/x_img_test.pkl', 'wb'))
# pickle.dump(y_col_test, open('states/y_col_test.pkl', 'wb'))

# open
X_img = pickle.load(open('states/x_img.pkl', 'rb'))
y_col = pickle.load(open('states/y_col.pkl', 'rb'))
X_img_test = pickle.load(open('states/x_img_test.pkl', 'rb'))
y_col_test = pickle.load(open('states/y_col_test.pkl', 'rb'))

### Creating a ImageDataGenerator

In [7]:
# prepare data augmentation configuration
datagen = ImageDataGenerator(
        shear_range=0.2, rotation_range=10)

train_generator = datagen.flow(X_img, y_col)

test_generator = datagen.flow(X_img_test, y_col_test)

In [8]:
model = cnn.build_model(3)

starting model compile
model compile done


In [17]:
class TestModel(keras.callbacks.Callback):
    def __init__(self, test_gen):
        self.test_gen = test_gen
        self.test_scores = []

    def on_epoch_end(self, batch, logs={}):
        test_score = self.model.evaluate_generator(self.test_gen, 32)
        self.test_scores.append(test_score)
        print('\nTest accuracy score: {}'.format(test_score))

In [46]:
for i in range(2, 5):
    test_model = TestModel(test_generator)
    history = model.fit_generator(train_generator, nb_epoch=NB_EPOCH, samples_per_epoch=BATCH_SIZE, callbacks=[test_model])
    df_log = pd.DataFrame(history.history)
    df_log['test_diff'] = np.array(test_model.test_scores)[:, 0]
    df_log['test_acc'] = np.array(test_model.test_scores)[:, 1]
    df_log.to_csv('logs/log_{}.csv'.format(i))

Epoch 1/20
127/128 [============================>.] - ETA: 0s - loss: 0.1362 - acc: 0.8110

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '



Test accuracy score: [0.145894855260849, 0.9375]
159/128 [=====================================] - 6s - loss: 0.1355 - acc: 0.8365
Epoch 2/20
 96/128 [=====================>........] - ETA: 0s - loss: 0.1515 - acc: 0.7396
Test accuracy score: [0.16746312379837036, 0.8125]
128/128 [==============================] - 4s - loss: 0.1477 - acc: 0.7500     
Epoch 3/20
 96/128 [=====================>........] - ETA: 0s - loss: 0.1385 - acc: 0.8021
Test accuracy score: [0.14339485764503479, 0.96875]
128/128 [==============================] - 4s - loss: 0.1465 - acc: 0.8125     
Epoch 4/20
 96/128 [=====================>........] - ETA: 0s - loss: 0.1523 - acc: 0.8333
Test accuracy score: [0.13835629820823669, 1.0]
128/128 [==============================] - 5s - loss: 0.1520 - acc: 0.7969     
Epoch 5/20
 96/128 [=====================>........] - ETA: 0s - loss: 0.1492 - acc: 0.7917
Test accuracy score: [0.16893112659454346, 0.90625]
128/128 [==============================] - 4s - loss: 0.1472 

KeyboardInterrupt: 

In [48]:
cnn.save(model, 'color', 'states/c_model')

In [ ]:
from IPython.core.display import display, HTML
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
def display_color(rgb):
    rgb = [int(color) for color in rgb]
    print (rgb)
    hex_code = rgb_to_hex(tuple(rgb))
    display(HTML('<div style="width:140px; height:140px; background-color: {};"></div>'.format(hex_code)))